In [0]:
from pyspark.sql.functions import count, year, desc

#### Create games dataframe from delta table

In [0]:
games_df = spark.read.table("steam_processed.games")

In [0]:
display(games_df)

#### Select wanted columns with only year from release date

In [0]:
selected_games_df = games_df.select(
                                    games_df["appid"],
                                    year(games_df["release_date"]).alias("release_year"))

#### Create list of games by year

In [0]:
game_list_by_year = selected_games_df.groupBy("release_year").agg(count("appid").alias("games_per_year")).orderBy(desc("release_year")).filter("release_year >= 2010")

In [0]:
display(game_list_by_year)

#### Write to delta table

In [0]:
game_list_by_year.write.mode("overwrite").format("delta").saveAsTable("steam_presentation.game_list_by_year")

#### Test table

In [0]:
%sql
SELECT * FROM steam_presentation.game_list_by_year WHERE release_year >= 2010 ORDER BY release_year;